### XNOR

In [1]:
import torch.nn.functional as F
from torch_explain.nn.concepts import ConceptReasoningLayer, IntpLinearLayer1, IntpLinearLayer2, IntpLinearLayer3
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch_explain as te
from torch_explain import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import logging
import warnings
warnings.filterwarnings('ignore')
# Set the logging level for matplotlib to WARNING to suppress debug information
logging.getLogger('matplotlib').setLevel(logging.WARNING)


PATH = 'models/'

2024-07-21 01:22:53,344 - matplotlib - DEBUG - matplotlib data path: d:\Github\pytorch_explain\env\Lib\site-packages\matplotlib\mpl-data
2024-07-21 01:22:53,351 - matplotlib - DEBUG - CONFIGDIR=C:\Users\a_h9\.matplotlib
2024-07-21 01:22:53,381 - matplotlib - DEBUG - interactive is False
2024-07-21 01:22:53,382 - matplotlib - DEBUG - platform is win32
2024-07-21 01:22:53,461 - matplotlib - DEBUG - CACHEDIR=C:\Users\a_h9\.matplotlib
2024-07-21 01:22:53,465 - matplotlib.font_manager - DEBUG - Using fontManager instance from C:\Users\a_h9\.matplotlib\fontlist-v390.json
2024-07-21 01:22:53,832 - matplotlib.pyplot - DEBUG - Loaded backend module://matplotlib_inline.backend_inline version unknown.
2024-07-21 01:22:53,836 - matplotlib.pyplot - DEBUG - Loaded backend inline version unknown.


In [3]:
from torch_explain.datasets import xnor
x, c, y = xnor(1000)

In [4]:
y = F.one_hot(y.long().ravel()).float()

DCR

In [5]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = ConceptReasoningLayer(embedding_size, y.shape[1])
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [6]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_DCRBase_XNOR.pth')))

<All keys matched successfully>

In [7]:
cem = model[0]
dcr = model[1]
cem.eval()
dcr.eval()
c_emb , c_pred = cem(x)
y_pred, sign_attn, filter_attn = dcr(c_emb, c_pred, return_attn=True)


In [8]:
dcr.explain( c_emb, c_pred, mode='global')

[{'class': 'y_0', 'explanation': '~c_0 & c_1', 'count': 264},
 {'class': 'y_0', 'explanation': 'c_0 & ~c_1', 'count': 239},
 {'class': 'y_1', 'explanation': 'c_0 & c_1', 'count': 259},
 {'class': 'y_1', 'explanation': '~c_0 & ~c_1', 'count': 237}]

LLR 1

In [9]:
isBias = True

In [10]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer1(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [11]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR1_XNOR.pth')))

<All keys matched successfully>

In [12]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [13]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([1000, 2]), torch.Size([1000, 2, 2]), torch.Size([1000, 2]))

In [14]:
dataframes = [pd.DataFrame() for _ in range(2)]
for i in range(2):
    dataframes[i]['c_0'] = c[:,0].detach().numpy()
    dataframes[i]['c_1'] = c[:,1].detach().numpy()
    dataframes[i]['w_0'] = weights[:,0,i].detach().numpy()
    dataframes[i]['w_1'] = weights[:,1,i].detach().numpy()
    dataframes[i]['bias'] = biases[:,i].detach().numpy()
    dataframes[i]['y'] = y[:,i].detach().numpy()

In [15]:
dataframes[0].head()

,c_0,c_1,w_0,w_1,bias,y
0,0.0,1.0,0.998515,0.999135,8.847340,1.0
1,1.0,1.0,0.110439,0.141643,-6.663466,0.0
2,0.0,0.0,0.008346,0.034460,-11.639477,0.0
3,0.0,1.0,0.989222,0.999738,9.082058,1.0
4,1.0,1.0,0.009883,0.818256,-6.966159,0.0


LLR 2

In [16]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer2(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [17]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR2_XNOR.pth')))

<All keys matched successfully>

In [18]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [19]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([1000, 2]), torch.Size([1000, 4, 2]), torch.Size([1000, 2]))

In [20]:
dataframes = [pd.DataFrame() for _ in range(2)]
for i in range(2):
    dataframes[i]['c_0'] = c[:,0].detach().numpy()
    dataframes[i]['c_1'] = c[:,1].detach().numpy()
    dataframes[i]['w_0'] = weights[:,0,i].detach().numpy()
    dataframes[i]['w_1'] = weights[:,1,i].detach().numpy()
    dataframes[i]['bias'] = biases[:,i].detach().numpy()
    dataframes[i]['y'] = y[:,i].detach().numpy()

In [21]:
dataframes[0].head()

,c_0,c_1,w_0,w_1,bias,y
0,0.0,1.0,0.999936,0.964901,8.706429,1.0
1,1.0,1.0,0.018338,0.005407,-7.551785,0.0
2,0.0,0.0,0.009999,0.013659,-10.772019,0.0
3,0.0,1.0,0.999695,0.999906,12.336782,1.0
4,1.0,1.0,0.003017,0.046243,-7.127800,0.0


LLR 3

In [22]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 16),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(16, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer3(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [23]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR3_XNOR.pth')))

<All keys matched successfully>

In [24]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [25]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([1000, 2]), torch.Size([1000, 2, 2]), torch.Size([1000, 2]))

In [26]:
dataframes = [pd.DataFrame() for _ in range(2)]
for i in range(2):
    dataframes[i]['c_0'] = c[:,0].detach().numpy()
    dataframes[i]['c_1'] = c[:,1].detach().numpy()
    dataframes[i]['w_0'] = weights[:,0,i].detach().numpy()
    dataframes[i]['w_1'] = weights[:,1,i].detach().numpy()
    dataframes[i]['bias'] = biases[:,i].detach().numpy()
    dataframes[i]['y'] = y[:,i].detach().numpy()

In [27]:
dataframes[0].head()

,c_0,c_1,w_0,w_1,bias,y
0,0.0,1.0,0.991679,0.990943,11.005300,1.0
1,1.0,1.0,-0.333780,-0.970270,-6.472853,0.0
2,0.0,0.0,-0.937669,-0.850431,-8.911013,0.0
3,0.0,1.0,0.974302,0.999573,13.893045,1.0
4,1.0,1.0,-0.950076,-0.519914,-6.498202,0.0


_____